# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [10]:
# Write your code below.

%load_ext dotenv

%dotenv ../01_materials/labs/.env



The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [11]:
import dask
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [24]:
# Write your code below.
import os
from glob import glob

PRICE_DATA = os.getenv("PRICE_DATA")
print(PRICE_DATA)

parquet_files = glob(os.path.join(PRICE_DATA, "*/*/*.parquet"))
#print(parquet_files)

dd_px = dd.read_parquet(parquet_files).set_index("ticker")
dd_px.head()

../../05_src/data/prices/


,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year
ticker,,,,,,,,,,
A,2000-01-03,56.330471,56.464592,48.193848,51.502148,43.613007,4674353,Health Care,Life Sciences Tools & Services,2000
A,2000-01-04,48.730328,49.266811,46.316166,47.567955,40.281467,4765083,Health Care,Life Sciences Tools & Services,2000
A,2000-01-05,47.389126,47.567955,43.141991,44.617310,37.782822,5758642,Health Care,Life Sciences Tools & Services,2000
A,2000-01-06,44.080830,44.349072,41.577251,42.918453,36.344166,2534434,Health Care,Life Sciences Tools & Services,2000
A,2000-01-07,42.247852,47.165592,42.203148,46.494991,39.372864,2819626,Health Care,Life Sciences Tools & Services,2000


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [33]:
# Write your code below.

# print(dd_px.head())

dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag = x['Close'].shift(1),
                       Adj_Close_lag = x['Adj Close'].shift(1))
).assign(
    returns = lambda x: x['Adj Close']/x['Adj_Close_lag'] - 1
).assign(
    hi_lo_range = lambda x: x['High'] - x['Low']
))





C:\Users\admin\AppData\Local\Temp\ipykernel_19660\1699903059.py:5: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [34]:
# Write your code below.

import pandas as pd

df = dd_feat.compute()

df['Rolling_Average'] = df['returns'].rolling(10).mean()
df['Rolling_Average'] 



ticker
OXY          NaN
OXY          NaN
OXY          NaN
OXY          NaN
OXY          NaN
          ...   
INTU    0.008518
INTU    0.010734
INTU    0.011941
INTU    0.009628
INTU    0.010109
Name: Rolling_Average, Length: 2746333, dtype: float64

In [37]:
print(df)

             Date        Open        High         Low       Close   Adj Close  \
ticker                                                                          
OXY    2003-01-02   13.556569   13.939794   13.556569   13.935003    7.948109   
OXY    2003-01-03   13.891891   13.987697   13.863149   13.958955    7.961768   
OXY    2003-01-06   13.891891   14.164938   13.891891   14.107454    8.046469   
OXY    2003-01-07   13.963745   13.982906   13.614053   13.647585    7.784178   
OXY    2003-01-08   13.527827   13.724230   13.460763   13.585311    7.748655   
...           ...         ...         ...         ...         ...         ...   
INTU   2024-06-24  630.719971  633.719971  622.460022  623.880005  623.880005   
INTU   2024-06-25  629.169983  630.549988  624.390015  630.469971  630.469971   
INTU   2024-06-26  628.000000  638.650024  624.489990  637.479980  637.479980   
INTU   2024-06-27  639.080017  657.159973  635.510010  651.440002  651.440002   
INTU   2024-06-28  652.00000

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?

No, it is not necessary to convert a Dask DataFrame to a pandas DataFrame to calculate the moving average return. I believe many of the operations available in pandas are also available in Dask.


+ Would it have been better to do it in Dask? Why?

Yes. The Dask DataFrames are designed to handle operations on large datasets that may not fit into memory so it would have better performance when datasets are huge.

(1 pt)

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Calculations         |Calculations were done correctly.|Calculations were not done correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.